
# Práctica 2 — Bibliotecas de Procesamiento y Análisis de Datos en Python

**Objetivo:** Familiarizarse con **Pandas**, **NumPy** y **Dask** para manipular, analizar y procesar datos (incluyendo datos grandes) de manera eficiente.

> **Estructura del notebook:** Cada sección incluye un **título**, **instrucciones** y **celdas de código** listas para ejecutar.



## Parte 1 — Preparación del entorno

### 1.1 Instalación de bibliotecas


In [12]:
# pip install --upgrade pip
# pip install pandas numpy "dask[dataframe]" jupyter




### 1.2 Verificación de instalación

python -m pip list


In [14]:

import pandas as pd
import numpy as np
import dask.dataframe as dd

print("Todas las bibliotecas se han cargado correctamente:",
      f"Pandas {pd.__version__} / NumPy {np.__version__}")


Todas las bibliotecas se han cargado correctamente: Pandas 2.3.3 / NumPy 2.3.4



## Parte 2 — Manipulación de datos con Pandas

> En esta sección trabajaremos con **`empleados.csv`** (archivo pequeño).

### 2.1 Cargar un Conjunto de Datos


In [16]:

# 2.1 Carga del dataset
import pandas as pd

ruta_empleados = "empleados.csv"
df = pd.read_csv(ruta_empleados)

print("Dataset cargado. Dimensiones:", df.shape)
df.head()


Dataset cargado. Dimensiones: (10, 6)


,nombre,edad,departamento,salario_mensual,fecha_contratacion,estado_civil
0,Juan Pérez,28,Recursos Humanos,4500,2019-05-10,Casado
1,María López,34,Marketing,5200,2016-03-15,Soltero
2,Carlos Ramírez,42,Finanzas,6100,2012-11-22,Casado
3,Ana González,25,Desarrollo,4800,2020-01-03,Soltero
4,Lucía Fernández,29,Soporte Técnico,4300,2018-07-19,Casado



### 2.2 Análisis exploratorio de Datos (EDA)
- Descripción de los datos
- Visualización de datos faltantes


In [19]:
# Descripción estadística del conjunto de datos
print(df.describe())

            edad  salario_mensual
count  10.000000        10.000000
mean   32.800000      5160.000000
std     6.713171       965.171257
min    25.000000      4000.000000
25%    28.250000      4525.000000
50%    31.000000      4850.000000
75%    36.250000      5875.000000
max    45.000000      7000.000000


In [20]:

# Conteo de valores faltantes
df.isnull().sum()


nombre                0
edad                  0
departamento          0
salario_mensual       0
fecha_contratacion    0
estado_civil          0
dtype: int64


### 2.3 Limpieza de datos
- Eliminación de nulos
- Eliminación de duplicados


In [28]:
# Eliminar filas con datos faltantes
df_cleaned = df.dropna()

# Eliminar filas duplicadas
df_cleaned = df_cleaned.drop_duplicates()

print("Datos limpios:")
print(df_cleaned.head())


Datos limpios:
            nombre  edad      departamento  salario_mensual  \
0       Juan Pérez    28  Recursos Humanos             4500   
1      María López    34         Marketing             5200   
2   Carlos Ramírez    42          Finanzas             6100   
3     Ana González    25        Desarrollo             4800   
4  Lucía Fernández    29   Soporte Técnico             4300   

  fecha_contratacion estado_civil  
0         2019-05-10       Casado  
1         2016-03-15      Soltero  
2         2012-11-22       Casado  
3         2020-01-03      Soltero  
4         2018-07-19       Casado  



### 2.4 Transformación de datos
- Crear columna `salario_anual = salario_mensual * 12`


In [30]:
# Agregar una columna de salario anual basado en el salario mensual
df_cleaned['salario_anual'] = df_cleaned['salario_mensual'] * 12
print(df_cleaned[['nombre', 'salario_mensual', 'salario_anual']].head())

            nombre  salario_mensual  salario_anual
0       Juan Pérez             4500          54000
1      María López             5200          62400
2   Carlos Ramírez             6100          73200
3     Ana González             4800          57600
4  Lucía Fernández             4300          51600


## Parte 3 — Procesamiento numérico con NumPy

### 3.1 Crear Arrays NumPy

In [41]:
# Crear arrays de NumPy
salario_mensual = np.array(df_cleaned['salario_mensual'])
bono_anual = np.array([1000, 1200, 800, 950, 1100])  # ejemplo de bonificaciones

# Calcular ingresos para los primeros 5 empleados (coincide con el tamaño del bono)
total_ingresos = salario_mensual[:bono_anual.size] * 12 + bono_anual
print('Total de ingresos anuales con bonificación:', total_ingresos)


Total de ingresos anuales con bonificación: [55000 63600 74000 58550 52700]


### 3.2 Operaciones Matemáticas Avanzadas

In [39]:
# Estadísticas básicas
print('Salario promedio:', np.mean(salario_mensual))
print('Mediana del salario:', np.median(salario_mensual))
print('Desviación estándar del salario:', np.std(salario_mensual))

Salario promedio: 5160.0
Mediana del salario: 4850.0
Desviación estándar del salario: 915.6418513807678



## Parte 4 — Procesamiento de Grandes Volúmenes de Datos con Dask


### 4.1 Cargar un gran conjunto de da Datos con Dask

> Primero **generaremos** un archivo grande `empleados_grandes.csv` y luego lo **cargaremos con Dask**.


In [44]:
# Definir parámetros
n = 10000 # Número de registros

# Crear datos aleatorios
np.random.seed(0)

nombres = ['Juan Pérez', 'María López', 'Carlos Ramírez', 'Ana González',
           'Lucía Fernández', 'Luis Martínez', 'Patricia Gómez', 'Jorge Sánchez',
           'Andrea Pérez', 'Pedro Ruiz', 'Juan García', 'Carla Mena',
           'Raúl Torres', 'Marta Rivera']
departamentos = ['Recursos Humanos', 'Marketing', 'Finanzas', 'Desarrollo',
'Soporte Técnico']
estados_civiles = ['Casado', 'Soltero', 'Viudo', 'Divorciado']

# Generar DataFrame con datos aleatorios
data = {
    'nombre': np.random.choice(nombres, n),
    'edad': np.random.randint(22, 60, size=n),
    'departamento': np.random.choice(departamentos, n),
    'salario_mensual': np.random.randint(3500, 8000, size=n),
    'fecha_contratacion': pd.to_datetime(np.random.choice(pd.date_range('2005-01-01', '2023-01-01'), n)),
    'estado_civil': np.random.choice(estados_civiles, n)
}

df_grande = pd.DataFrame(data)

# Guardar a un archivo CSV
df_grande.to_csv('empleados_grandes.csv', index=False)
print("Archivo 'empleados_grandes.csv' generado con éxito, con", len(df_grande), "registros")

Archivo 'empleados_grandes.csv' generado con éxito, con 10000 registros



### 4.1 Cargar el CSV grande con Dask


In [45]:
# Crear un DataFrame Dask a partir de un archivo CSV
ddf = dd.read_csv('empleados_grandes.csv')

# Mostrar las primeras filas
ddf.head()

,nombre,edad,departamento,salario_mensual,fecha_contratacion,estado_civil
0,Raúl Torres,59,Recursos Humanos,6857,2016-06-21,Casado
1,Luis Martínez,36,Finanzas,4349,2017-02-23,Soltero
2,Juan Pérez,24,Finanzas,3579,2007-03-04,Soltero
3,Ana González,38,Marketing,7457,2015-05-08,Casado
4,Carla Mena,53,Desarrollo,5946,2015-12-26,Casado



### 4.2 Procesamiento Distribuido de Datos


In [47]:

# Media (promedio) de salario_mensual
mean_salary = ddf['salario_mensual'].mean().compute()

print("Salario mensual promedio (calculado con Dask):", mean_salary)


Salario mensual promedio (calculado con Dask): 5737.713


### 4.3 Aplicar Operaciones de Filtrado y Agregación

In [48]:
# Filtrar empleados con salario > 5000
high_salary = ddf[ddf['salario_mensual'] > 5000]

# Contar el número de empleados con salarios altos
num_high_salary = high_salary.shape[0].compute()

print("Número de empleados con salario mayor a 5000:", num_high_salary)

Número de empleados con salario mayor a 5000: 6615



## Parte 5 — Conclusión y reflexión

---
**Preguntas guía:**

- ¿Qué beneficios ofrece el uso de herramientas como Pandas, NumPy y Dask en el análisis de datos?

    El uso de herramientas como Pandas, NumPy y Dask ofrece grandes ventajas en el análisis de datos.  
    Pandas facilita la manipulación, limpieza y transformación de datos mediante estructuras como los DataFrames, que permiten trabajar con información tabular de forma intuitiva y eficiente.  
    NumPy proporciona operaciones matemáticas y estadísticas de alto rendimiento gracias a sus arreglos multidimensionales (arrays), optimizados en memoria y velocidad.  
    Por su parte, Dask amplía la capacidad de Pandas y NumPy, permitiendo procesar conjuntos de datos más grandes que la memoria del equipo, distribuyendo el trabajo en múltiples núcleos o máquinas.  
    En conjunto, estas bibliotecas reducen el tiempo de desarrollo, mejoran la eficiencia del análisis y hacen posible manejar datos desde pequeños hasta masivos con facilidad.

- ¿Cómo se pueden aplicar estas herramientas en escenarios del mundo real?

    Estas herramientas se aplican ampliamente en entornos reales de ciencia de datos, análisis empresarial y aprendizaje automático.  
    Por ejemplo, Pandas se usa en el análisis de ventas y comportamiento de clientes, NumPy en modelos estadísticos y cálculos numéricos, y Dask en proyectos de Big Data donde se requiere procesar millones de registros provenientes de sensores, redes sociales o transacciones financieras.  
    También son útiles en la limpieza de datos para modelos de inteligencia artificial, la optimización de procesos industriales y la predicción de tendencias en sectores como salud, educación, finanzas o energía.  
    En resumen, permiten transformar grandes volúmenes de datos en información útil para la toma de decisiones estratégicas.

---
**Preguntas de discusión**

- ¿Cuáles son las limitaciones de Pandas y NumPy en comparación con Dask
para el procesamiento de Big Data?

    Las principales limitaciones de Pandas y NumPy son su dependencia de la memoria RAM y su ejecución en un solo hilo de procesamiento (single-thread).  
    Aunque son muy eficientes con conjuntos de datos medianos, su rendimiento disminuye o se vuelve inviable con volúmenes muy grandes de información.  
    En cambio, Dask permite dividir los datos en fragmentos más pequeños y procesarlos en paralelo, aprovechando múltiples núcleos o equipos, lo que lo hace ideal para el procesamiento distribuido y análisis de Big Data.


- ¿Qué consideraciones se deben tener en cuenta al elegir una herramienta
para un proyecto de análisis de datos?

    Al elegir una herramienta, se deben considerar:

    - Tamaño del conjunto de datos: Pandas y NumPy son ideales para datos pequeños o medianos; Dask para volúmenes grandes.  
    - Recursos del sistema: Si se cuenta con memoria limitada o se requiere escalabilidad, Dask es la mejor opción.  
    - Tipo de análisis: NumPy es óptimo para cálculos numéricos; Pandas para manipulación tabular; Dask para procesamiento paralelo.  
    - Facilidad de uso y rendimiento: Es importante equilibrar la complejidad del entorno con la velocidad y precisión requerida.  

    En conclusión, la elección depende del contexto del proyecto, el tamaño de los datos y los objetivos del análisis.